In [ ]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [4]:
df = pd.read_csv('/content/drive/MyDrive/4.2/soft computing/Sentiment Analysis/cleaned_news.csv')


In [5]:
df = df.sample(frac=1)
df.tail(10)

,Unnamed: 0,Comment,Tag,cleaned,Length
6967,6967,আর্ন্তজাতিক বাজারে এর দাম ৫০০ ডলারের নিচে এখা...,0,আর্ন্তজাতিক বাজারে দাম ৫০০ ডলারের নিচে এতো দাম,8
18614,18614,শেষটায় আর চোখে পানি ধরে রাখা গেল না। ফুলির অভ...,1,শেষটায় চোখে পানি ফুলির অভিনয় কথায় চোখ ভিজে গ্র...,19
19448,19448,চরম একটা নাটক। যারা আমার মতো apurbo -এর fan ...,1,চরম একটা নাটক মতো জানাও,5
15130,15130,জাস্টফ্রেন্ড বা বেষ্টফ্রেন্ড নাম হলে আরও সুন্...,1,জাস্টফ্রেন্ড বেষ্টফ্রেন্ড নাম সুন্দর শেষে নিশু...,9
14227,14227,বাংলাদেশের খেলাটা দেখে আসতেছি ভাই। বসের নাটক ...,1,বাংলাদেশের খেলাটা আসতেছি ভাই বসের নাটক ত ভালোয়...,9
19245,19245,Hats off really!!! appreciable যাদের বিবেক না...,1,বিবেক ফরজে আইন অন্যায় সহ্য মেয়ে এভাবে মাঠে নাম...,18
7366,7366,সারাবছরে এতগুলো বল খেলার কেউ কি আছে বাংলাদেশে?...,2,সারাবছরে এতগুলো বল খেলার বাংলাদেশে প্লিস বলুন,7
18214,18214,দারুন নাটক টা.......,1,দারুন নাটক টা,3
8663,8663,রোভমান পাওয়েল এর আউট ফিরিয়ে নেয়া উচিত ছিল। এই...,0,রোভমান পাওয়েল আউট ফিরিয়ে নেয়া দায়িত্বটি বাংলাদ...,14
17640,17640,কিছুই বলার নেই...Just Awesome!!!,1,বলার,1


In [6]:
from transformers import pipeline

gpt2_bengali = pipeline('text-generation',model="flax-community/gpt2-bengali", tokenizer='flax-community/gpt2-bengali')

Downloading:   0%|          | 0.00/864 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76M [00:00<?, ?B/s]

In [7]:
df['Tag'].value_counts()

1    9113
0    8233
2    2373
Name: Tag, dtype: int64

In [8]:
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [9]:
df['Comment'].iloc[0]

'@ রাজিবফেসবুক, অনলাইন ছেড়ে রাস্তায় নামুন! ইসি, সিইসি, প্রধানমন্ত্রীর পদত্যাগ ঘটান! আপনাকে স্বাগত!'

In [10]:
token = tokenizer.encode_plus(
    df['Comment'].iloc[0],
    max_length = 256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensor='tf'
)

Keyword arguments {'return_tensor': 'tf'} not recognized.


In [11]:
x_input_ids = np.zeros((len(df),256))
x_attention_mask = np.zeros((len(df),256))


In [12]:
def generate_training_data(df,ids,masks,tokenizer):
  for i, text in tqdm(enumerate(df['Comment'])):
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=256,
        truncation=True,
        padding = 'max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    ids[i, :] = tokenized_text.input_ids
    masks[i,:]= tokenized_text.attention_mask
  return ids,masks

In [13]:
x_input_ids,x_attention_mask=generate_training_data(df,x_input_ids,x_attention_mask,tokenizer)

0it [00:00, ?it/s]

In [14]:
x_input_ids

array([[  101.,  1046., 19809., ...,     0.,     0.,     0.],
       [  101., 90130., 47721., ...,     0.,     0.,     0.],
       [  101.,  2857.,  2045., ...,     0.,     0.,     0.],
       ...,
       [  101., 14705.,  2392., ...,     0.,     0.,     0.],
       [  101.,  4972., 28579., ...,     0.,     0.,     0.],
       [  101.,  2178.,  7635., ...,     0.,     0.,     0.]])

In [15]:
labels = np.zeros((len(df),3))

In [16]:
labels[np.arange(len(df)),df['Tag'].values]=1

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((x_input_ids,x_attention_mask,labels))

In [18]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [19]:
def SentimentDatasetMapFunction(input_ids,attention_mask,labels):
  return{
      'input_ids':input_ids,
      'attention_mask':attention_mask
  }, labels

In [20]:
dataset = dataset.map(SentimentDatasetMapFunction)

In [21]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [22]:
dataset = dataset.shuffle(10000).batch(16,drop_remainder=True)

In [23]:
p=14
train_size = int((len(df)//16)*p)

In [24]:
train_size

17248

In [25]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [26]:
from transformers import TFBertModel

In [27]:
bert_Model = TFBertModel.from_pretrained('sagorsarker/bangla-bert-base')

Downloading:   0%|          | 0.00/976M [00:00<?, ?B/s]

Some layers from the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at sagorsarker/bangla-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [28]:
input_ids = tf.keras.layers.Input(shape=(256,),name='input_ids',dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,),name='attention_mask',dtype='int32')

bert_embds = bert_Model.bert(input_ids,attention_mask=attention_masks)[1]
intermediate_layer = tf.keras.layers.Dense(512,activation='relu',name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3,activation='softmax',name='output_layer')(intermediate_layer)

model = tf.keras.Model(inputs=[input_ids,attention_masks],outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  164396544   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [29]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5,decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()


In [30]:
model.compile(optimizer=optim,loss=loss_func,metrics=[acc,recall,precision])

In [31]:
hist = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose =1
)

Epoch 1/10
1232/1232 [==============================] - 1026s 824ms/step - loss: 0.6561 - accuracy: 0.7186 - recall: 0.6534 - precision: 0.7680
Epoch 2/10
1232/1232 [==============================] - 1016s 824ms/step - loss: 0.5321 - accuracy: 0.7772 - recall: 0.7241 - precision: 0.8223
Epoch 3/10
1232/1232 [==============================] - 1016s 824ms/step - loss: 0.4232 - accuracy: 0.8278 - recall: 0.7886 - precision: 0.8629
Epoch 4/10
1232/1232 [==============================] - 1016s 825ms/step - loss: 0.2943 - accuracy: 0.8843 - recall: 0.8631 - precision: 0.9061
Epoch 5/10
1232/1232 [==============================] - 1016s 825ms/step - loss: 0.1785 - accuracy: 0.9368 - recall: 0.9278 - precision: 0.9448
Epoch 6/10
1232/1232 [==============================] - 1016s 825ms/step - loss: 0.1114 - accuracy: 0.9617 - recall: 0.9584 - precision: 0.9651
Epoch 7/10
1232/1232 [==============================] - 1017s 825ms/step - loss: 0.0826 - accuracy: 0.9736 - recall: 0.9718 - precision:

In [32]:
model.save('sentiment_model')

In [33]:
loaded_model = tf.keras.models.load_model('sentiment_model')